# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import pandas as pd
from IPython.display import JSON #for nice JSON (or dict) display in JupyterLab (not currently supported in Jupyter Notebook)

In [2]:
# get network data
city_bike_networks = requests.get("http://api.citybik.es/v2/networks").json()

# view json in nice format
# JSON(city_bike_networks)

In [3]:
# Select city
city='Vancouver'

In [4]:
city_lookup = pd.json_normalize(city_bike_networks['networks'], max_level=1, errors='ignore')
city_lookup.head(2)

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.75000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.89455,10.546343,NaN,NaN,NaN,NaN,NaN


In [5]:
city_lookup[city_lookup['location.city']==city]['id']

241    mobibikes
Name: id, dtype: object

In [6]:
network_id = 'mobibikes'
network_json = requests.get("http://api.citybik.es/v2/networks/"+network_id).json()
JSON(network_json)

<IPython.core.display.JSON object>

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [7]:
stations = pd.json_normalize(network_json['network']['stations'], max_level=1, errors='ignore')
stations.head(5)

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.slots,extra.uid
0,14,21,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-02-13T19:22:31.595000Z,4,True,1676316003,17,True,True,35,0001
1,7,9,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-02-13T19:22:31.591000Z,1,True,1676315894,8,True,True,16,0004
2,13,13,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-02-13T19:22:31.592000Z,0,True,1676315968,13,True,True,26,0005
3,14,2,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-02-13T19:22:31.593000Z,0,True,1676316133,2,True,True,16,0007
4,7,9,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-02-13T19:22:31.604000Z,2,True,1676315653,7,True,True,16,0008


In [8]:
# df['team_count'] = df.groupby('team')['team'].transform('count')
stations['total_slots'] = stations['empty_slots']+stations['free_bikes']
stations.head(5)

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.slots,extra.uid,total_slots
0,14,21,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-02-13T19:22:31.595000Z,4,True,1676316003,17,True,True,35,0001,35
1,7,9,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-02-13T19:22:31.591000Z,1,True,1676315894,8,True,True,16,0004,16
2,13,13,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-02-13T19:22:31.592000Z,0,True,1676315968,13,True,True,26,0005,26
3,14,2,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-02-13T19:22:31.593000Z,0,True,1676316133,2,True,True,16,0007,16
4,7,9,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-02-13T19:22:31.604000Z,2,True,1676315653,7,True,True,16,0008,16


Put your parsed results into a DataFrame.

In [9]:
type(stations)

pandas.core.frame.DataFrame

In [10]:
stations.to_csv('stations.csv')